In [ ]:
#hide
#default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [ ]:
#export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *

from collections import defaultdict
from pprint import pformat
from inspect import signature,Parameter
import ast,contextlib,copy

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

## NotebookProcessor -

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')

In [ ]:
#export
def extract_comments(ss):
    "Take leading comments from lines of code in `ss`, remove `#`, and split"
    ss = ss.splitlines()
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or re.match('\s*[^#\s]', o))
    return L((s.strip()[1:]).strip().split() for s in ss[:first_code]).filter()

nbprocess comments start with `#`, followed by whitespace delimited tokens, which `extract_comments` extracts from the start of a cell, up until a blank line or a line containing something other than comments:

In [ ]:
exp  = """#export module
# hide
1+2
#bar"""
test_eq(extract_comments(exp), [['export', 'module'],['hide']])

In [ ]:
#export
class NotebookProcessor:
    "Base class for nbprocess notebook processors"
    def __init__(self, path, debug=False): self.nb,self.path,self.debug = read_nb(path),Path(path),debug

Subclass `NotebookProcessor` to add methods to act on nbprocess comments. The method names are of the form `cmd_type`, where "`cmd`" is the first word of the nbprocess comment, and `type` is the `cell_type` of the cell (normally "`code`"). The methods must take at least `comment` and `code` as params, plus extra params for any additional words included in a comment. Here's an example that prints any word following a "print me" comment:

In [ ]:
class _PrintExample(NotebookProcessor):
    def printme_code(self, to_print): print(to_print)

We can create a processor by passing it a notebook:

In [ ]:
everything_fn = '../tests/01_everything.ipynb'
proc = _PrintExample(everything_fn)

In [ ]:
#export
@functools.lru_cache(maxsize=None)
def _param_count(f):
    "Number of parameters accepted by function `f`"
    params = list(signature(f).parameters.values())
    # If there's a `*args` then `f` can take as many params as neede
    if first(params, lambda o: o.kind==Parameter.VAR_POSITIONAL): return 99
    return len([o for o in params if o.kind in (Parameter.POSITIONAL_ONLY,Parameter.POSITIONAL_OR_KEYWORD)])

The basic functionality of a notebook processor is to read and act on nbprocess comments.

In [ ]:
#export
@patch
def process_comment(self:NotebookProcessor, cell_type, comment, idx):
    cmd,*args = comment
    self.comment,self.idx = comment,idx
    cmd = f"{cmd}_{cell_type}"
    if self.debug: print(cmd, args)
    f = getattr(self, cmd, None)
    if not f or _param_count(f)<len(args): return True
    return f(*args)

Behind the scenes, `process_comment`  is used to call subclass methods. It is passed the comment (as a list split on spaces), the notebook cell itself, and the index number of the comment. You can subclass this to change the behavior of a processor.

In [ ]:
proc.process_comment("code",["printme","hello"], 0)

hello


If the wrong number of parameters are passed, it is silently ignored:

In [ ]:
proc.process_comment("code", ["printme","hello","there"], 0)

True

In [ ]:
#export
@patch
def process_cell(self:NotebookProcessor, cell):
    comments = extract_comments(cell.source)
    self.cell = cell
    if not comments: return
    keeps = [self.process_comment(cell.cell_type, comment, i)
             for i,comment in enumerate(comments)]
    self.cell.source = ''.join([o for i,o in enumerate(self.cell.source.splitlines(True))
                                if i>=len(keeps) or keeps[i]])

Subclass `process_cell` to change how `process_comment` is called. The return value of `process_cell` is used to replace the cell in the notebook.

In [ ]:
cell = make_code_cell("#printme hello\n1+1")
proc.process_cell(cell);

hello


Cell comments that have been processed are removed from the cell source, unless the processing function returns `True`:

In [ ]:
cell.source

'1+1'

In [ ]:
#export
@patch
def process(self:NotebookProcessor):
    "Process all cells with `process_cell` and replace `self.nb.cells` with result"
    for i in range_of(self.nb.cells): self.process_cell(self.nb.cells[i])

In [ ]:
proc.process()

testing


`NotebookProcessor.process` doesn't change a notebook or act on any comments, unless you subclass it.

In [ ]:
everything = read_nb(everything_fn)
proc = NotebookProcessor(everything_fn)
proc.process()
for a_,b_ in zip(everything.cells, proc.nb.cells): test_eq(str(a_),str(b_))

## ExportModuleProcessor -

In [ ]:
#export
class ExportModuleProcessor(NotebookProcessor):
    "A `NotebookProcessor` which exports code to a module"
    def __init__(self, path, dest, mod_maker=ModuleMaker, debug=False):
        dest = Path(dest)
        store_attr()
        super().__init__(path,debug=debug)

    def process(self):
        self.default_exp,self.modules,self.in_all = None,defaultdict(L),defaultdict(L)
        super().process()

Specify `path` containing the source notebook, `dest` where the module(s) will be exported to, and optionally a class to use to create the module (`ModuleMaker`, by default).

In [ ]:
proc = ExportModuleProcessor(everything_fn, 'tmp')

In [ ]:
#export
@patch
def default_exp_code(self:ExportModuleProcessor, exp_to): self.default_exp = exp_to

You must include a `default_exp` comment somewhere in your notebook to show what module to export to by default.

In [ ]:
proc.process()
test_eq(proc.default_exp, 'everything')

In [ ]:
#export
@patch
def exporti_code(self:ExportModuleProcessor, exp_to=None):
    "Export a cell, without including the definition in `__all__`"
    self.modules[ifnone(exp_to, '#')].append(self.cell)

Exported cells are stored in a `dict` called `modules`, where the keys are the modules exported to. Those without an explicit module are stored in the `'#'` key, which will be exported to `default_exp`.

In [ ]:
proc.process()
proc.modules['#']

(#1) [#just another comment
def c_y_nall(): ...]

In [ ]:
#export
@patch
def export_code(self:ExportModuleProcessor, exp_to=None):
    "Export a cell, adding the definition in `__all__`"
    self.exporti_code(exp_to)
    self.in_all[ifnone(exp_to, '#')].append(self.cell)

In [ ]:
proc.process()
print(proc.in_all['some.thing'])

[def h_n(): ...]


In [ ]:
#export
@patch
def create_modules(self:ExportModuleProcessor):
    "Create module(s) from notebook"
    self.process()
    for mod,cells in self.modules.items():
        all_cells = self.in_all[mod]
        name = self.default_exp if mod=='#' else mod
        mm = self.mod_maker(dest=self.dest, name=name, nb_path=self.path, is_new=mod=='#')
        mm.make(cells, all_cells)

Let's check we can import a test file:

In [ ]:
shutil.rmtree('tmp')
proc = ExportModuleProcessor('../tests/00_some.thing.ipynb', 'tmp')
proc.create_modules()

In [ ]:
g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a'])
test_eq(g['tmp'].some.thing.a, 1)

We'll also check that our 'everything' file exports correctly:

In [ ]:
proc = ExportModuleProcessor(everything_fn, 'tmp')
proc.create_modules()

g = exec_new('import tmp.everything; from tmp.everything import *')

_alls = L("a b d e m n o p q".split())
for s in _alls.map("{}_y"): assert s in g, s
for s in "c_y_nall _f_y_nall g_n h_n i_n j_n k_n l_n".split(): assert s not in g, s
for s in _alls.map("{}_y") + ["c_y_nall", "_f_y_nall"]: assert hasattr(g['tmp'].everything,s), s

That notebook should also export one extra function to `tmp.some.thing`:

In [ ]:
del(sys.modules['tmp.some.thing']) # remove from module cache
g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a','h_n'])
test_eq(g['tmp'].some.thing.h_n(), None)

In [ ]:
#export
def nb_export(nbname, lib_name=None):
    if lib_name is None: lib_name = get_config().lib_name
    ExportModuleProcessor('02_export.ipynb', lib_name).create_modules()

## Export -

In [ ]:
path = Path('../nbprocess')
(path/'export.py').unlink(missing_ok=True)

nb_export('02_export.ipynb', 'nbprocess')
g = exec_new('import nbprocess.export')
assert hasattr(g['nbprocess'].export, 'ModuleMaker')

In [ ]:
nb_export('00_read.ipynb', 'nbprocess')
nb_export('01_maker.ipynb', 'nbprocess')